In [1]:
import taichi as ti
from taichi import exp
from taichi.math import pi
ti.init(arch = ti.gpu, fast_math=True)

[Taichi] version 1.7.4, llvm 15.0.7, commit b4b956fd, osx, python 3.10.19
[Taichi] Starting on arch=metal


[I 11/09/25 03:50:50.864 134067] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
# Parameters to tune
n       = 500                   # The number of pixels in our image
c       = 4                     # Wavespeed
dx      = 1/n                   # Separation (we have normalized the length to be 1)
dt      = 1e-4                  # A way to calculate dt in the convergent region
h       = (c*dt/dx)**2 / 2
epsilon = 1e-4

# Structural taichi things
u_curr  = ti.field(ti.f32, (n,n))
u_curr.fill(0)
u_prev  = ti.field(ti.f32, (n,n))
u_prev.fill(0)

# Auxiliary vairiables for inversion
r       = ti.field(ti.f32, (n,n))
r.fill(0)
p       = ti.field(ti.f32, (n,n))
p.fill(0)
Dp      = ti.field(ti.f32, (n,n))
Dp.fill(0)
b       = ti.field(ti.f32, (n,n))
b.fill(0)

pixels  = ti.Vector.field(3, ti.f32, (n,n))                             # Colors to display
window  = ti.ui.Window("2D Waves", res=(n, n), fps_limit=400)
gui = window.get_canvas()

@ti.kernel
def initialize(x:ti.f32, y:ti.f32, sx:ti.f32, sy:ti.f32):
    for i,j in u_curr:
        if i!=0 and j!=0 and i!=n-1 and j!=n-1: 
            v = 200*exp(-((2*i-n)/n - x)**2/(2*sx**2) - ((2*j-n)/n - y)**2/(2*sy**2))#/(2*pi*sx*sy)
            u_curr[i,j] = v * dt
        else:
            u_curr[i,j] = 0

@ti.func
def calc_lhs(u_c:ti.template(), u_p:ti.template(), u_out:ti.template(), h:ti.f32):
    for i,j in u_out:
        if i!=0 and j!=0 and i!=n-1 and j!=n-1: 
            u_out[i,j] = h*(u_c[i+1,j] + u_c[i-1,j] + u_c[i,j+1] + u_c[i,j-1]) + (2 - 4*h)*u_c[i,j] - u_p[i,j]
        else:
            u_curr[i,j] = 0

@ti.func
def apply_D(u_in:ti.template(), u_out:ti.template(), h:ti.f32):
    for i,j in u_out:
        if i!=0 and j!=0 and i!=n-1 and j!=n-1: 
            u_out[i,j] = (1+4*h)*u_in[i,j] - h*(u_in[i+1,j] + u_in[i-1,j] + u_in[i,j+1] + u_in[i,j-1])
        else:
            u_curr[i,j] = 0

@ti.func
def dot(u:ti.template(), v:ti.template()) -> ti.f32:
    S = 0.
    for i,j in u:
        S += u[i,j]*v[i,j]
    return S

@ti.func
def set(u:ti.template(), v:ti.template()):
    for i,j in u:
        u[i,j] = v[i,j]

@ti.func
def add(u:ti.template(), v:ti.template(), out:ti.template(), a:ti.f32): 
    for i,j in out:
        out[i,j] = u[i,j] + a*v[i,j]

@ti.kernel
def step(h:float, epsilon:float, max_iter:int):
    # Calculate the left hand side of Du = b
    calc_lhs(u_curr, u_prev, b, h)
    # u_prev = u_curr
    set(u_prev, u_curr)

    # calculate r and p
    apply_D(u_curr, r, h)
    add(b, r, r, -1)
    set(p, r)
    norm_r = dot(r,r)
    # converged=False

    # Loop while small solutions are there to be found
    for k in ti.static(range(1)):
        # if converged: continue

        apply_D(p,Dp,h)
        A = dot(p,Dp)
        
        # if abs(A) <= 1e-15: converged=True

        A = norm_r/A

        add(u_curr, p, u_curr, A)
        add(r, Dp, r, -A)

        B = 1/norm_r
        norm_r = dot(r,r)
        B *= norm_r

        # if norm_r > epsilon: converged=True
        add(r,p,p,B)    


@ti.kernel
def draw():
    for i,j in pixels:
        pixels[i,j] = u_curr[i,j]

initialize(0,0,1e-1,1e-1)

while window.running:    
    draw()

    step(h,epsilon,100)

    gui.set_image(pixels)
    window.show()

In [3]:
# 